In [1]:
%load_ext autoreload
%autoreload 2

Gorth16 in zkp-playground
====================

## Bilinear groups

We will work over bilinear groups $(p,\mathbb{G}_1,\mathbb{G}_2,\mathbb{G}_T,e,g,h)$ with the following properties:

* $\mathbb{G}_1, \mathbb{G}_2, mathbb{G}_T$ are groups of prime order p

* The pairing $e$: $\mathbb{G}_1x\mathbb{G}_2 \rightarrow \mathbb{G}_T$ is a binlinear map

* $g$ is a generator for $\mathbb{G}_1$, $h$ is a generator for $\mathbb{G}_2$ and $e(g, h)$ is a generator for $\mathbb{G}_T$

* There are efficient algorithms for computing group operations, evaluating the bilinearmap, deciding membership of the groups, deciding equality of group elements andsampling generators of the groups. We refer to these as the generic group operations.

In [2]:
from zkp_playground.curves.barreto_naehrig.bn128 import BN128FP as FP, BN128FP2 as FP2, BN128FP12 as FP12, ECGBN128 as ECG
from zkp_playground.algebra.utils import randfield

In [3]:
g = ECG.G1
h = ECG.G2

We write $[a]_1$ for $g^a$, $[b]_2$ for $h^b$, and $[c]_T$ for $e(g, h)^c$. A vector of group elements will be represented as $[\mathbf{a}]_i$. We defined dot product as $[\mathbf{a}]_1 \circ [\mathbf{b}]_2 = [\mathbf{a}\circ \mathbf{b}]_T$

## NIZK

let $R$ be a relation generator that given a security parameter $λ$ in unary returns a polynomial time decidable binary relation $R$. For pairs $(\phi, \omega) \in R$.

For pairs $(\phi, \omega) \ in R$, we call $\phi$ the statement and $\omega$ the witness. We defined $R_{\lambda}$  to be the set of possible relations $R $the relation gen-erator may output given $1^λ$. The relation generator may also output someside information, an auxiliary input $z$, which will be given to the adversary.

An efficient prover publicly verifiable non-interactive argument for $R$ is a quadruple of probabilistic polynomial algorithms $(Setup,Prove,Vfy,Sim)$ such that

* $(\sigma,\tau) \leftarrow Setup(R)$: The setup produces a common reference stringσand a simulationtrapdoorτfor the relationR

* $\pi \leftarrow Prov(R, \sigma, \phi, \omega)$:The prover algorithm takes as input a common reference string σ and $(φ,w)∈R$ and returns an argument π.

* $0/1 \leftarrow Vfy(R, \sigma, \phi, \pi)$: The  verification  algorithm  takes  as  input  a  common  referencestring σ, a statement φ and an argumentπand returns 0 (reject) or 1 (accept).

* $\pi \leftarrow Sim(R, \tau, \phi)$: The simulator takes as input a simulation trapdoor and statementφand returns an argument π.

## QAP

Given $n$ equations we pick arbitrary distinct $r_1,\cdots,r_n \in \mathbb{F}$ and define
$$
t(x) = \prod_{q=1}^n (x - r_q)
$$

$$
\sum_{i=0}^m a_i u_i(X) \circ \sum_{i=0}^m a_i v_i(X) = \sum_{i=0}^m a_i w_i(X) + h(X)t(X)
$$

 we will be working with quadratic arithmetic programsRthat have thefollowing description
 
 $$
 R = (p, \mathbb{G}_1,\mathbb{G}_2, \mathbb{G}_T, e, g, h, l, \{u_i(X), v_i(X), w_i(X)\}_{i=0}^n, t(X))
 $$

with $|p|=\lambda$. The realation defineds a field $\mathbb{Z}_p$ and a language of statement $(a_i,\cdots, a_l) \in \mathbb{Z}_p^l$ and witness $(a_{l+1}, \cdots, a_m) \in \mathbb{Z}_p^{m-l}$ such that with $a_0=1$

In [4]:
from collections import namedtuple

RationalGenerator = namedtuple("RelationGenerator", ["F", "G1", "G2", "Gt", "e", "g", "h", "l", "U", "V", "W", "T"])

In [5]:
from zkp_playground.zkp.r1cs import R1CS
from zkp_playground.zkp.qap import QAP


In [6]:
@R1CS.r1cs(FP)
def f(x, k, c):
    y = x + c + k
    return y ** 3


In [7]:
qap = QAP(f.A, f.B, f.C)
U, V, W, T = qap.qap
a = f.witness(FP(89), FP(8), FP(8))

In [8]:
len(a)

8

In [9]:
R = RationalGenerator(FP, ECG, ECG, ECG, ECG.e, ECG.G1, ECG.G2, 4, U, V, W, T)

## $(\sigma,\tau) \leftarrow Setup(R)$:

Pick $\alpha, \beta, \gamma, \delta, x \leftarrow \mathbb{Z}_P^*$, Define $\tau = (\alpha, \beta, \gamma, \delta, x)$ and compute $\sigma = ([\mathbf{\sigma_1}]_1, [\mathbf{\sigma_2}]_2$ where

$$
(\mathbf{\sigma}_1, \mathbf{\sigma}_2) = \left( \left(\begin{split}
\alpha, \beta, \delta, \{x^i\}_{i=0}^{n-1},
\left\{\frac{\beta u_i(x) + \alpha v_i (x) + w_i (x)}{\gamma} \right\}_{i=0}^l \\
\left\{ \frac{\beta u_i (x) + \alpha v_i(x) + w_i(x)}{\delta} \right\}_{i=l+1}^m,
\left\{\frac{x^i t(x)}{\delta} \right \}_{i=0}^{n-2}
\end{split}\right) , \left(\beta, \gamma, \delta, \{x^i\}^{n-1}_{i=0} \right)
\right)
$$

In [10]:
from typing import Iterable, Tuple, Type
from operator import add
from functools import reduce

def setup(R, m) -> Tuple[Iterable, Tuple[R.G1, R.G2]]:
    tau = alpha, beta, delta, gamma, x = \
        randfield(R.F), randfield(R.F), randfield(R.F), randfield(R.F), randfield(R.F)
    n = R.U[0].degree
    sigma_1 = [alpha, beta, delta] + \
              [x ** i for i in range(0, n)] + \
              [(R.U[i](x) * beta + R.V[i](x) * alpha + R.W[i](x)) / gamma  for i in range(0, R.l)] + \
              [(R.U[i](x) * beta + R.V[i](x) * alpha + R.W[i](x)) / delta  for i in range(R.l, m)] + \
              [(x ** i * R.T(x)) / delta for i in range(0, n-1)]
    sigma_2 = [beta, gamma, delta] + \
              [x**i for i in (0, n)]
    
    sigma = ([R.g @ s for s in sigma_1], [R.h @ s for s in sigma_2])
    return tau, sigma

In [11]:
tau, sigma = setup(R, len(a))

## $\pi \leftarrow Prov(R, \tau, \sigma, a_1, \cdots, a_m)$

Pick $r, s \leftarrow \mathbb{Z}_p$ and compute $\pi = ([A]_1, [C]_1, [B]_2$ where

$$
A = \alpha + \sum_{i=0}^m a_i u_i(x) + r\delta
$$

$$
B = \beta + \sum_{i=0}^m a_i v_i(x) + s\delta
$$

$$
C = \frac{\sum_{i=l+1}^m a_i(\beta u_i(x) + \alpha v_i(x) + w_i(x)) + h(x)t(x)}{\delta} + As + Br - rs\delta
$$

In [12]:
H = qap.H(a)
r, s = randfield(R.F), randfield(R.F)

In [13]:
def prov(R, tau, sigma, a, r=None, s=None) -> Tuple[R.G1, R.G1, R.G2]:
    r = r or randfield(R.F)
    s = s or randfield(R.F)
    alpha, beta, delta, gamma, x = tau
    m = len(a)
    A = alpha + reduce(add, [a[i] * (R.U[i](x)) for i in range(0, m)]) + r * delta
    B = beta + reduce(add, [a[i] * (R.V[i](x)) for i in range(0, m)]) + s * delta
    C = (reduce(add, [a[i]*(beta * R.U[i](x) + alpha * R.V[i](x) + R.W[i](x)) for i in range(R.l, m)]) + H(x) * R.T(x)) / delta \
        + (A * s + B * r - r * s * delta)
    return (R.g @ A, R.g @ C, R.h @ B)
    
    

    

In [14]:
pi = prov(R, tau, sigma, a, r, s)
pi

(ECGBN128::(BN128FP::281310725290666030206366069317222884517966662075706611566485183815163112332, BN128FP::541395234746644492961869912378820509015814567228705139788995088820815380843),
 ECGBN128::(BN128FP::21674701336930079718699836919101012933310610226916221837212176739600678116454, BN128FP::3910949947198748869737832941355822079718355141175871797899075959619262502889),
 ECGBN128::(BN128FP2::[BN128FP::10554078494814173896352153020611964781690436455454402770033426366954360885030, BN128FP::13948166196858543184115853215378799493814149634918174810126119853058155200177], BN128FP2::[BN128FP::7068156277073286431755378318831850182213699935107164712926020763518941543754, BN128FP::16977681661032304994362014249913566848822132944048783556149813069918307688961]))

## $0/1 \leftarrow Vfy(R, \tau, a_1, \cdots, a_l, \pi)$

Parse $\pi = ([A]_1, [C]_1, [B]_2 \in G_1^2 x G_2$. Accept the proof iff:

$$
[A]_1 \circ [B]_2 = [\alpha]_1 \circ [\beta]_2 + \sum_{i=0}^l a_i \left[ \frac{\beta u_i(x) + \alpha v_i(x) + w_i(x)}{\gamma} \right]_1 \circ [\gamma]_2 + [C]_1 \circ [\delta]_2
$$

In [43]:
def vfy(R, tau, a, pi):
    alpha, beta, delta, gamma, x = tau
    A_1, C_1, B_2 = pi
    lhs = R.e(A_1, B_2)
    D = R.g @ (reduce(add, [
                a[i] * (beta * R.U[i](x) + alpha * R.V[i](x) + R.W[i](x))
                for i in range(0, R.l)]) / gamma)
    rhs = R.e(R.g @ alpha, R.h @ beta) + \
          R.e(D, R.h @ gamma) + \
          R.e(C_1, R.h @ delta)
    return lhs == rhs

In [16]:
vfy(R, tau, a, pi)

False

## Explain

On $Prov$ phase:


\begin{aligned}
A &= \alpha + \sum_{i=0}^m a_i u_i(x) + r\delta \\
&= \alpha + A_{qap} + r\delta
\end{aligned}



\begin{aligned}
C &= \frac{\sum_{i=l+1}^m a_i(\beta u_i(x) + \alpha v_i(x) + w_i(x)) + h(x)t(x)}{\delta} + As + Br - rs\delta\\
&= \frac{1}{\delta} \sum_{i=l+1}^m \left(a_i(\beta u_i(x) + \alpha v_i(x) + w_i(x)) + h(x)t(x)\right) + As + Br - rs\delta \\ 
&=\frac{\beta}{\delta}\sum_{i=l+1}^m a_i u_i(x) + \frac{\alpha}{\delta} \sum_{i=l+1}^m a_i v_i(x) +\frac{1}{\delta} \sum_{i=l+1}^m w(x)t(x)+
\frac{1}{\delta} \sum_{i=l+1}^m h(x)t(x) + As + Br - rs \delta \\
&=\frac{\beta}{\delta}A_{qap_{l+1, m}}+ \frac{\alpha}{\delta} B_{qap_{l+1, m}} +
\frac{1}{\delta} H_{qap_{l+1, m}}T_{qap_{l+1, m}} + s\alpha + sA_{qap} + sr\delta + r\beta + B_{qap}(x) + rs\delta - rs \delta \\
&=\frac{\beta}{\delta}A_{qap_{l+1, m}}+ \frac{\alpha}{\delta} B_{qap_{l+1, m}} + \frac{1}{\delta} C_{qap_{l+1, m}} +
\frac{1}{\delta} H_{qap_{l+1, m}}T_{qap_{l+1, m}} + s\alpha + r\beta + sA_{qap} + rB_{qap} + sr\delta
\end{aligned}

In [35]:
A_1, C_1, B_2 = prov(R, tau, sigma, a, r, s)
alpha, beta, delta, gamma, x = tau

In [46]:
A_qap, B_qap, C_qap, T_qap, H_qap = qap.proof(x, a)
assert A_qap * B_qap == C_qap + T_qap * H_qap

In [23]:
A_qap_l2m, B_qap_l2m, C_qap_l2m, T_qap_l2m, H_qap_l2m = qap.proof(x, a, R.l, len(a))

In [24]:
A_qap_02l, B_qap_02l, C_qap_02l, T_qap_02l, H_qap_02l = qap.proof(x, a, 0, R.l)

In [25]:
assert A_qap_l2m + A_qap_02l == A_qap

In [39]:
assert A_1 == R.g @ (alpha + A_qap + r * delta)
assert B_2 == R.h @ (beta + B_qap + s * delta)
assert C_1 == R.g @ (beta/delta * A_qap_l2m + alpha/delta * B_qap_l2m + C_qap_l2m / delta + T_qap_l2m * H_qap_l2m / delta + \
s * alpha + r * beta + s * A_qap + r * B_qap + s * r * delta)

On Vry phase, On RHS


\begin{aligned}
Rhs &= [\alpha]_1 \circ [\beta]_2 + \sum_{i=0}^l a_i \left[ \frac{\beta u_i(x) + \alpha v_i(x) + w_i(x)}{\gamma} \right]_1 \circ [\gamma]_2 + [C]_1 \circ [\delta]_2
\end{aligned}


let $D$ be:

\begin{aligned}
D &= \sum_{i=0}^l a_i \left[ \frac{\beta u_i(x) + \alpha v_i(x) + w_i(x)}{\gamma} \right]_1\\
&= \left[ \frac{\beta}{\gamma} A_{qap_{0,l}} + \frac{\alpha}{\gamma} B_{qap_{0,l}} + \frac{1}{\gamma}C_{qap{0,l}} \right]_1
\end{aligned}


on LHS

\begin{aligned}
Lhs &= [A]_1 \circ [B]_2 \\
    &= [\alpha + A_{qap} + r\delta]_1 \circ [\beta + B_{qap} + s\delta]_2\\
    &= [\alpha]_1 \circ [\beta]_2 + [\alpha]_1 \circ [B_{qap}]_2 + [\alpha]_1 \circ [s\delta]_2 +
        [A_{qap}]_1 \circ [\beta]_2 + [A_{qap}]_1 \circ [B_{qap}]_2 + [A_{qap}]_1 \circ [s\delta]_2 + 
        [r\delta]_1 \circ [\beta]_2 + [r\delta]_1 \circ [B_{qap}]_2 + [r\delta]_1 \circ [s\delta]_2      \\   
\end{aligned}



In [30]:
D = R.g @ ((beta * A_qap_02l / gamma) + (alpha * B_qap_02l / gamma) + (C_qap_02l / gamma))

In [42]:
R.e(R.g@alpha, R.h@beta) + R.e(D, R.h@gamma) + R.e(C_1, R.h@delta)

BN128FP12::[BN128FP::9330449265283841176651423630334483389881271944891602221114362827661055476293, BN128FP::3354409673755371983280255478421534248748733526128671779037339027769562921653, BN128FP::9032943893224560533813494353778033815265280494725552701223466306532562408363, BN128FP::20088728969940215183321814935550029996608041242640605356704549980047320518049, BN128FP::18030228504134844122494323508821756770511416763855727297876063120856341757160, BN128FP::169325934333137158517685284296403162501205545486135580450364020437116714121, BN128FP::2845441260958762360823588804277393235774535289370341647579920801708029399089, BN128FP::10940039976724277233478837743677779394199086163351327945629536143944039064233, BN128FP::7038628046749229038243203404847452197603537491890897146413361893444672993311, BN128FP::13886748842474463072057568402134672461509900194438327696809994126109587422429, BN128FP::20703668335252942867379546492016929329676903203769776526305103410617787659489, BN128FP::467321677322339454

In [33]:
R.e(A_1, B_2)

BN128FP12::[BN128FP::17252391017434909372428091732675408363546428331052505370006235005243072106548, BN128FP::9482680340098112675210043059897047779410721310912427267073025212303117051848, BN128FP::1979584153737159189781096664810374928769307629354083262267236814150536292558, BN128FP::18553690925490717331955593639751088143014200052200032708180322291330312954472, BN128FP::3422840256232188361602700148153052192378175106399788917506939063277417346537, BN128FP::1639772524891448783297385090513301389765667623937208567483411831082273837215, BN128FP::8141520494985817575748668702412180678900365908129992123055634895095155403782, BN128FP::16886973294915502824782251771281740500390285925622576687967426891145831086648, BN128FP::16890753527222819075534913824361172222899002510613833924976229985315094619348, BN128FP::19886816564075747290106381107593544762203390373908994317863546846690349303720, BN128FP::21314553503412483507970416454110849304489072462045768696711860285228335516595, BN128FP::1217298451293811